### 네이버 뉴스 섹션별 TOPIC 크롤링


In [1]:
from urllib.request import urlopen
import requests
import pandas as pd
from bs4 import BeautifulSoup


In [2]:
#02_0에서 저장한 섹션별 link 파일 불러오기

df_menu=pd.read_csv('./crawl_data/navew_news_section.csv',index_col=0) #제목 오타
df_menu.head()

,section,link
0,언론사별,https://news.naver.com/?viewType=pc
1,정치,https://news.naver.com/main/main.naver?mode=LS...
2,경제,https://news.naver.com/main/main.naver?mode=LS...
3,사회,https://news.naver.com/main/main.naver?mode=LS...
4,생활/문화,https://news.naver.com/main/main.naver?mode=LS...


### 경제 섹션을 예시 코드로 작성

In [3]:
df_menu['link'][2]

'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101'

In [4]:
#경제 섹션 소스 요청
url=df_menu['link'][2]

# res=requests.get(url) #요청 후 응답객체 반환
#ConnectionResetError
# ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))
#원격 봇이라고 생각하고 연결 종료

# urlopen()함수는 진행됌
res=urlopen(url)
res.read()

b'\r\n\r\n\r\n\r\n<!DOCTYPE HTML>\r\n<html lang="ko">\r\n<head>\r\n<meta charset="euc-kr">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta name="referrer" contents="always">\r\n<meta http-equiv="refresh" content="600" />\r\n<meta name="viewport" content="width=1106" />\r\n\r\n    \r\n    \r\n        \r\n            \r\n                \r\n                    \r\n                    \r\n                    \r\n                    \r\n                        \r\n\t                        \r\n\t\t                    \r\n\t\t                        \r\n\t\t                        \r\n\t\t                        \r\n\t\t                    \r\n\t\t                    \r\n\t\t                    \r\n\t\t                    \r\n\t\t                    \r\n\t\t                    \r\n\t\t                \r\n                    \r\n                    \r\n                \r\n            \r\n            \r\n            \r\n            \r\n            \r\n        \r\n    \r\n   

위 코드처럼 자동화 봇으로 보고 연결 끊는 경우 해결방법.  
    - 서버에 요청시 header를 구성해 추가해 줌 즉, bot이 아님을 증명  
    - header 확인  
         - 브라우저에서 해당 사이트 접속 시 생성하는 헤더를  
        - 아래와 같이 개발자도구로 확인  


In [4]:
#소스 가져오기
url = df_menu['link'][2]
headers={'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}

res=requests.get(url,headers=headers)


In [5]:
html = res.text

In [6]:
bs_obj = BeautifulSoup(html,'html.parser')

In [7]:
bs_obj


<!DOCTYPE HTML>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta contents="always" name="referrer"/>
<meta content="600" http-equiv="refresh">
<meta content="width=1106" name="viewport">
<meta content="경제 : 네이버 뉴스" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://news.naver.com/main/main.naver?mode=LSD&amp;mid=shm&amp;sid1=101" property="og:url"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" property="og:image">
<meta content="증권, 금융, 부동산, 기업, 국제 등 경제 분야 뉴스 제공" property="og:description"/>
<meta content="네이버" property="og:article:author">
<meta content="summary" name="twitter:card"/>
<meta content="경제 : 네이버 뉴스" name="twitter:title"/>
<meta content="네이버 뉴스" name="twitter:site"/>
<meta content="네이버 뉴스" name="twitter:creator"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" name="tw

- 네이버 뉴스 섹션을 확인하면 각 섹션별 topic이 있고 topic중 헤드라인이 노출
- 섹션에서 수집하는 것 보다 토픽페이지로 이동해서 토픽기사 전체를 크롤링 하면 더 많은 기사를 추출할 수 있음
- 섹션 페이지 에서는 토픽의 title과 link를 크롤링
- 단, 정치 섹션은 구성이 약간 다르므로 수정해서 사용


In [8]:
#토픽 추출
##main_content > div:nth-child(1) > div > div > h2 > span:nth-child(2)
#h2태그의 class : cluster_head_topic

topic_list=bs_obj.findAll("h2",{"class":"cluster_head_topic"})
len(topic_list)


12

In [9]:
topic_list

[<h2 class="cluster_head_topic">
 <a class="nclicks(cls_eco.clstitle)" href="/main/clusterArticles.naver?id=c_202207310930_00000002&amp;mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=366&amp;aid=0000831106">
 <span class="cluster_head_sub_topic">CU 할인바다 행사 개최 </span><span class="cluster_head_sub_topic">물가인상 걱정 줄인다 </span>
 </a>
 </h2>,
 <h2 class="cluster_head_topic">
 <a class="nclicks(cls_eco.clstitle)" href="/main/clusterArticles.naver?id=c_202207221010_00000017&amp;mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=008&amp;aid=0004777275">
 <span class="cluster_head_sub_topic">스타벅스 발암물질 책임론 들썩 </span><span class="cluster_head_sub_topic">"다른 기업 굿즈는 괜찮겠죠?" </span>
 </a>
 </h2>,
 <h2 class="cluster_head_topic">
 <a class="nclicks(cls_eco.clstitle)" href="/main/clusterArticles.naver?id=c_202207311210_00000005&amp;mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=014&amp;aid=0004875599">
 <span class="cluster_head_sub_topic">김학도 중진공 이사장 </span><span class="cluster_head_sub_topic">"투자 귀재" 짐 로저스 만나 </span>
 </

In [10]:
topic_list[0]

<h2 class="cluster_head_topic">
<a class="nclicks(cls_eco.clstitle)" href="/main/clusterArticles.naver?id=c_202207310930_00000002&amp;mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=366&amp;aid=0000831106">
<span class="cluster_head_sub_topic">CU 할인바다 행사 개최 </span><span class="cluster_head_sub_topic">물가인상 걱정 줄인다 </span>
</a>
</h2>

In [12]:
topic_list[0].find('a')

<a class="nclicks(cls_eco.clstitle)" href="/main/clusterArticles.naver?id=c_202207310930_00000002&amp;mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=366&amp;aid=0000831106">
<span class="cluster_head_sub_topic">CU 할인바다 행사 개최 </span><span class="cluster_head_sub_topic">물가인상 걱정 줄인다 </span>
</a>

In [13]:
topic_list[0].find('a')['href']

'/main/clusterArticles.naver?id=c_202207310930_00000002&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000831106'

In [17]:
topic_list[0].find('a').text.replace("\n","")

'CU 할인바다 행사 개최 물가인상 걱정 줄인다 '

In [18]:
#전체 토픽 title과 link 반복 출력
for topic in topic_list :
    a_t=topic.find('a') #a태그 없는 문구도 있음 -> 에러발생->예외처리 필요
    
    print(a_t.text.replace('\n',''))
    print(a_t['href'])
    
#마지막 부분에서 a태그가 없는 요소(빈 item)가 있어서 에러 발생
#예외처리 진행

CU 할인바다 행사 개최 물가인상 걱정 줄인다 
/main/clusterArticles.naver?id=c_202207310930_00000002&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000831106
스타벅스 발암물질 책임론 들썩 "다른 기업 굿즈는 괜찮겠죠?" 
/main/clusterArticles.naver?id=c_202207221010_00000017&mode=LSD&mid=shm&sid1=101&oid=008&aid=0004777275
김학도 중진공 이사장 "투자 귀재" 짐 로저스 만나 
/main/clusterArticles.naver?id=c_202207311210_00000005&mode=LSD&mid=shm&sid1=101&oid=014&aid=0004875599
휘발유 1800원대 진입 눈앞 4주 연속 하락세 
/main/clusterArticles.naver?id=c_202207300610_00000025&mode=LSD&mid=shm&sid1=101&oid=056&aid=0011311890
블루리본 보험설계사 2415명 선정 손보협회 
/main/clusterArticles.naver?id=c_202207311210_00000007&mode=LSD&mid=shm&sid1=101&oid=016&aid=0002022527
자영업자 3명 중 1명 폐업 몰리는 자영업자들 
/main/clusterArticles.naver?id=c_202207311110_00000007&mode=LSD&mid=shm&sid1=101&oid=023&aid=0003707139
9년 연속 시공능력평가 1위 삼성물산 
/main/clusterArticles.naver?id=c_202207311110_00000001&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000831082
캐스퍼 결함신고 90여건 "에어컨·열선 켜면 덜덜" 
/main/clusterArticles.naver?id=c_202207310920_

AttributeError: 'NoneType' object has no attribute 'text'

In [19]:
# 전체 토픽 title과 link 출력
for topic in topic_list :
    try :
        a_t=topic.find('a') #a태그 없는 문구도 있음 -> 에러발생->예외처리 필요

        print(a_t.text.replace('\n',''))
        print(a_t['href'])
    except :
        print("에러발생")


CU 할인바다 행사 개최 물가인상 걱정 줄인다 
/main/clusterArticles.naver?id=c_202207310930_00000002&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000831106
스타벅스 발암물질 책임론 들썩 "다른 기업 굿즈는 괜찮겠죠?" 
/main/clusterArticles.naver?id=c_202207221010_00000017&mode=LSD&mid=shm&sid1=101&oid=008&aid=0004777275
김학도 중진공 이사장 "투자 귀재" 짐 로저스 만나 
/main/clusterArticles.naver?id=c_202207311210_00000005&mode=LSD&mid=shm&sid1=101&oid=014&aid=0004875599
휘발유 1800원대 진입 눈앞 4주 연속 하락세 
/main/clusterArticles.naver?id=c_202207300610_00000025&mode=LSD&mid=shm&sid1=101&oid=056&aid=0011311890
블루리본 보험설계사 2415명 선정 손보협회 
/main/clusterArticles.naver?id=c_202207311210_00000007&mode=LSD&mid=shm&sid1=101&oid=016&aid=0002022527
자영업자 3명 중 1명 폐업 몰리는 자영업자들 
/main/clusterArticles.naver?id=c_202207311110_00000007&mode=LSD&mid=shm&sid1=101&oid=023&aid=0003707139
9년 연속 시공능력평가 1위 삼성물산 
/main/clusterArticles.naver?id=c_202207311110_00000001&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000831082
캐스퍼 결함신고 90여건 "에어컨·열선 켜면 덜덜" 
/main/clusterArticles.naver?id=c_202207310920_

- base_rul : https://news.naver.com
- sub_url : /main/clusterArticles.naver?id=c_202207251130_00000383&mode=LSD&mid=shm&sid1=101&oid=648&aid=0000009369

- 수집된 url은 기본 url이 빠져 있으므로 기본 url을 첨부하여 수집해야 함

In [20]:
#link에 기본 url 추가
base_url ='https://news.naver.com'

# 전체 토픽 title과 link 출력
for topic in topic_list :
    try :
        a_t=topic.find('a') #a태그 없는 문구도 있음 -> 에러발생->예외처리 필요

        print(a_t.text.replace('\n',''))
        print(base_url + a_t['href'])
    except :
        print("에러발생")


CU 할인바다 행사 개최 물가인상 걱정 줄인다 
https://news.naver.com/main/clusterArticles.naver?id=c_202207310930_00000002&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000831106
스타벅스 발암물질 책임론 들썩 "다른 기업 굿즈는 괜찮겠죠?" 
https://news.naver.com/main/clusterArticles.naver?id=c_202207221010_00000017&mode=LSD&mid=shm&sid1=101&oid=008&aid=0004777275
김학도 중진공 이사장 "투자 귀재" 짐 로저스 만나 
https://news.naver.com/main/clusterArticles.naver?id=c_202207311210_00000005&mode=LSD&mid=shm&sid1=101&oid=014&aid=0004875599
휘발유 1800원대 진입 눈앞 4주 연속 하락세 
https://news.naver.com/main/clusterArticles.naver?id=c_202207300610_00000025&mode=LSD&mid=shm&sid1=101&oid=056&aid=0011311890
블루리본 보험설계사 2415명 선정 손보협회 
https://news.naver.com/main/clusterArticles.naver?id=c_202207311210_00000007&mode=LSD&mid=shm&sid1=101&oid=016&aid=0002022527
자영업자 3명 중 1명 폐업 몰리는 자영업자들 
https://news.naver.com/main/clusterArticles.naver?id=c_202207311110_00000007&mode=LSD&mid=shm&sid1=101&oid=023&aid=0003707139
9년 연속 시공능력평가 1위 삼성물산 
https://news.naver.com/main/clusterArticles.nave

In [28]:
### 수집한 data를 DataFrame으로 생성

In [29]:
#수집 내용 list에 저장
topic_title =[]
topic_link=[]

In [30]:
#link에 기본 url 추가
base_url ='https://news.naver.com'

# 전체 토픽 title과 link 출력
for topic in topic_list :
    try :
        a_t=topic.find('a') 
        topic_title.append(a_t.text.replace('\n',''))
        topic_link.append(base_url + a_t['href'])
    except :
        print("에러발생")

에러발생


In [31]:
topic_title

['CU 할인바다 행사 개최 물가인상 걱정 줄인다 ',
 '스타벅스 발암물질 책임론 들썩 "다른 기업 굿즈는 괜찮겠죠?" ',
 '김학도 중진공 이사장 "투자 귀재" 짐 로저스 만나 ',
 '휘발유 1800원대 진입 눈앞 4주 연속 하락세 ',
 '블루리본 보험설계사 2415명 선정 손보협회 ',
 '자영업자 3명 중 1명 폐업 몰리는 자영업자들 ',
 '9년 연속 시공능력평가 1위 삼성물산 ',
 '캐스퍼 결함신고 90여건 "에어컨·열선 켜면 덜덜" ',
 '법인세 중간예납 8월 말까지 내달 31일까지 신고·납부 ',
 '"하도급 갑질" 홍성건설에 과징금 2700만원 공정위 ',
 '공매도 위반해도 소액 과태료 올 들어 최저 ']

In [24]:
topic_link

['https://news.naver.com/main/clusterArticles.naver?id=c_202207310930_00000002&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000831106',
 'https://news.naver.com/main/clusterArticles.naver?id=c_202207221010_00000017&mode=LSD&mid=shm&sid1=101&oid=008&aid=0004777275',
 'https://news.naver.com/main/clusterArticles.naver?id=c_202207311210_00000005&mode=LSD&mid=shm&sid1=101&oid=014&aid=0004875599',
 'https://news.naver.com/main/clusterArticles.naver?id=c_202207300610_00000025&mode=LSD&mid=shm&sid1=101&oid=056&aid=0011311890',
 'https://news.naver.com/main/clusterArticles.naver?id=c_202207311210_00000007&mode=LSD&mid=shm&sid1=101&oid=016&aid=0002022527',
 'https://news.naver.com/main/clusterArticles.naver?id=c_202207311110_00000007&mode=LSD&mid=shm&sid1=101&oid=023&aid=0003707139',
 'https://news.naver.com/main/clusterArticles.naver?id=c_202207311110_00000001&mode=LSD&mid=shm&sid1=101&oid=366&aid=0000831082',
 'https://news.naver.com/main/clusterArticles.naver?id=c_202207310920_00000117&mode=LSD&mi

In [25]:
topic_df=pd.DataFrame({'contents':topic_title,"URL":topic_link})
topic_df.head()

,contents,URL
0,CU 할인바다 행사 개최 물가인상 걱정 줄인다,https://news.naver.com/main/clusterArticles.na...
1,"스타벅스 발암물질 책임론 들썩 ""다른 기업 굿즈는 괜찮겠죠?""",https://news.naver.com/main/clusterArticles.na...
2,"김학도 중진공 이사장 ""투자 귀재"" 짐 로저스 만나",https://news.naver.com/main/clusterArticles.na...
3,휘발유 1800원대 진입 눈앞 4주 연속 하락세,https://news.naver.com/main/clusterArticles.na...
4,블루리본 보험설계사 2415명 선정 손보협회,https://news.naver.com/main/clusterArticles.na...


### 위 내용을 결합해서 함수로 구성
- 다른 섹션에도 적용하기 위해 
- 함수명 : get_topic(url)
- 반환 : 섹션별 수집 데이터를 dict로 구성 후 반환 

In [32]:
#함수 구성
def get_topic(url,section) :
    
    #수집 내용 list에 저장
    topic_title = []
    topic_link = []
    
    #소스 요청해서 가져오기
    headers={'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
    res=requests.get(url,headers=headers)
    
    #파서기 생성(bs4)
    html = res.text
    bs_obj = BeautifulSoup(html,'html.parser')
    
    #토픽 추출
    # h2태그의 class:cluster_head_topic
    topic_list=bs_obj.findAll("h2",{"class":"cluster_head_topic"})

    #최종 수집 항목을 list에 저장
    # link에 기본 url 추가
    base_url = 'https://news.naver.com'
    # 전체 토픽 title과 link 출력
    for topic in topic_list :
        try : 
            a_t = topic.find('a')
            topic_title.append(a_t.text.replace('\n',''))
            topic_link.append(base_url + a_t['href'])
        except :
            print('에러발생')

    return({"topic":topic_title, "url": topic_link, "section":section})

In [27]:
#함수 테스트
get_topic(url,'경제')

에러발생


{'topic': ['SK이노베이션 "ESG 리포트" 발간 "이해관계자 눈높이 맞춘다" ',
  '스타벅스 발암물질 책임론 들썩 "다른 기업 굿즈는 괜찮겠죠?" ',
  '국토부 "GTX 추진단" 발족 "개통 앞당기고 신설 검토" ',
  '법인세 중간예납 8월 말까지 내달 31일까지 신고·납부 ',
  '수입농산물 유통이력관리 품목 확대 대추·생강 4종 추가 ',
  '포스코청암재단 포스코히어로즈 5명 선정 흉기 난동범 제압한 이웃 ',
  '9년 연속 시공능력평가 1위 삼성물산 ',
  '정부 "외국인 총수 지정" 재검토 통상마찰 소지 차단 관건 ',
  '휘발유 1800원대 진입 눈앞 4주 연속 하락세 ',
  '자영업자 3명 중 1명 폐업 몰리는 자영업자들 ',
  '5년새 33% 증가 청년층 다중채무액 158조 원 '],
 'url': ['https://news.naver.com/main/clusterArticles.naver?id=c_202207310910_00000551&mode=LSD&mid=shm&sid1=101&oid=018&aid=0005280595',
  'https://news.naver.com/main/clusterArticles.naver?id=c_202207221010_00000017&mode=LSD&mid=shm&sid1=101&oid=081&aid=0003291187',
  'https://news.naver.com/main/clusterArticles.naver?id=c_202207311110_00000005&mode=LSD&mid=shm&sid1=101&oid=056&aid=0011312052',
  'https://news.naver.com/main/clusterArticles.naver?id=c_202207311210_00000022&mode=LSD&mid=shm&sid1=101&oid=056&aid=0011312069',
  'https://news.naver.com/main/clusterArticles.naver?id

In [33]:
# 02_0에서 저장한 섹션별 url 파일 가져오기
df_menu = pd.read_csv('./crawl_data/navew_news_section.csv', index_col=0)
df_menu.head(8)

,section,link
0,언론사별,https://news.naver.com/?viewType=pc
1,정치,https://news.naver.com/main/main.naver?mode=LS...
2,경제,https://news.naver.com/main/main.naver?mode=LS...
3,사회,https://news.naver.com/main/main.naver?mode=LS...
4,생활/문화,https://news.naver.com/main/main.naver?mode=LS...
5,IT/과학,https://news.naver.com/main/main.naver?mode=LS...
6,세계,https://news.naver.com/main/main.naver?mode=LS...
7,랭킹,https://news.naver.com/main/ranking/popularDay...


In [34]:
# 함수 test
pd.DataFrame(get_topic(df_menu['link'][3],df_menu['section'][3])) #사회에 대한 기사, 1 정치로 호출 불가

에러발생


,topic,url,section
0,"세대공존형 주택단지 은평혁신파크에 짓는다 오세훈표 ""고품질 임대주택"" 구체화",https://news.naver.com/main/clusterArticles.na...,사회
1,법원 부당 인사 동료와 다퉈 정직받은 직원,https://news.naver.com/main/clusterArticles.na...,사회
2,저소득층 기저귀·조제분유 지원금 다음 달부터 인상,https://news.naver.com/main/clusterArticles.na...,사회
3,사적 연금소득에도 건보료 피부양자 요건도 반영 검토,https://news.naver.com/main/clusterArticles.na...,사회
4,오후 9시까지 7만191명 확진 전주의 1.2배 많아,https://news.naver.com/main/clusterArticles.na...,사회
5,태풍 송다 영향권 제주·남해안에 많은 비,https://news.naver.com/main/clusterArticles.na...,사회
6,반도체 특화단지 조성 추진 인천시 K-반도체 중심지 도약,https://news.naver.com/main/clusterArticles.na...,사회
7,"""6호 태풍 트라세"" 발생 제주로 직진",https://news.naver.com/main/clusterArticles.na...,사회
8,보령 앞바다서 해양사고 잇따라 인명피해 없어,https://news.naver.com/main/clusterArticles.na...,사회
9,차량 7대 연쇄 추돌 연쇄추돌 일으킨 만취운전 20대男,https://news.naver.com/main/clusterArticles.na...,사회


## 추출하려하는 모든 섹션에 적용

In [35]:
#모든 섹션의 topic을 하나의 dataframe에 저장하기 위해 빈 dataframe을 생성
dict_sub={}
dict_sub['topic']=[]
dict_sub['url']=[]
dict_sub['section']=[]

topic_df=pd.DataFrame(dict_sub)
topic_df

,topic,url,section


In [43]:
#섹션별 헤드라인 추출
for i in range(1,7) : 
    temp = pd.DataFrame(get_topic(df_menu['link'][i],df_menu['section'][i]))
    topic_df = pd.concat([topic_df, temp], axis =0, ignore_index=True)

에러발생
에러발생
에러발생
에러발생
에러발생
에러발생
에러발생


In [44]:
# topic_df.head(10)
topic_df.tail(10)

,topic,url,section
122,예쁘긴 한데 뭐가 달라졌지? 윤곽 드러나는 갤Z플립4·폴드4,https://news.naver.com/main/clusterArticles.na...,IT/과학
123,"포로수용소 ""누가 포격"" 공방 UN-적십자사에 ""우크라 수형소 공격"" 조사 요청",https://news.naver.com/main/clusterArticles.na...,세계
124,中 창정5B 로켓 잔해 필리핀 남서부 바다로 추락,https://news.naver.com/main/clusterArticles.na...,세계
125,라트비아에 가스 공급 중단 러 국영 가스프롬,https://news.naver.com/main/clusterArticles.na...,세계
126,"""올해 곡물 수확량 평년 절반"" 불과할 듯 ""조만간 수출 시작""",https://news.naver.com/main/clusterArticles.na...,세계
127,中 제조업 경기 한달만에 위축 국면 재진입,https://news.naver.com/main/clusterArticles.na...,세계
128,"바이든 코로나19 재확진 주치의 ""증상은 없어""",https://news.naver.com/main/clusterArticles.na...,세계
129,추가 피해 우려 켄터키 홍수 사망자 25명으로 늘어,https://news.naver.com/main/clusterArticles.na...,세계
130,"뉴욕시 ""원숭이두창 비상사태"" 선포 아프리카 밖 처음",https://news.naver.com/main/clusterArticles.na...,세계
131,"대만 中 침공대비 훈련 끝나자 ""불 타 죽는다""",https://news.naver.com/main/clusterArticles.na...,세계


#위 결과에는 정치 섹션이 들어오지 않음 -태그 구성이 다르기 때문
-정치 섹션도 포함되게 변경
### 위 결과에는 정치 섹션이 들어오지 않음 - 태그 구성이 다르기 때문
- 정치 섹션도 포함되게 함수 변경
- https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100 를 확인하면 topic이 따로 없음


In [ ]:
# #함수 구성
# def get_topic(url,section) :
    
#     #수집 내용 list에 저장
#     topic_title = []
#     topic_link = []
    
#     #소스 요청해서 가져오기
#     headers={'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
#     res=requests.get(url,headers=headers)
    
#     #파서기 생성(bs4)
#     html = res.text
#     bs_obj = BeautifulSoup(html,'html.parser')
    
#     #section 이 정치인 경우는 다른 태그를 추출
#     if section == '정치' :
#         topic_list = bs_obj.findAll('div',{'class':'cluster_foot_inner'})
        

#     else :
#         #토픽 추출
#         # h2태그의 class:cluster_head_topic
# class:cluster_head_topic

# topic_list=bs_obj.findAll("h2",{"class":"cluster_head_topic"})

#     #최종 수집 항목을 list에 저장
#     # link에 기본 url 추가
#     base_url = 'https://news.naver.com'
#     # 전체 토픽 title과 link 출력
#     for topic in topic_list :
#         try : 
#             a_t = topic.find('a')
#             topic_title.append(a_t.text.replace('\n',''))
#             topic_link.append(base_url + a_t['href'])
#         except :
#             print('에러발생')

#     return({"topic":topic_title, "url": topic_link, "section":section})

In [38]:
#선생님 코드
# 함수 구성
def get_topic(url,section) :
    
    # 수집 내용 list에 저장
    topic_title = []
    topic_link = []
    
    # 소스 요청해서 가져오기
    headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
    res = requests.get(url, headers=headers)
    
    # 파서기 생성(bs4 객체 생성)
    html = res.text
    bs_obj = BeautifulSoup(html,'html.parser')
    
    # section이 정치인 경우는 다른 태그를 추출
    if section == '정치' :
        topic_list = bs_obj.findAll('div',{'class':'cluster_foot_inner'})
    else :     
        # 토픽 추출
        # h2태그의 class:cluster_head_topic
        topic_list=bs_obj.findAll("h2",{"class":"cluster_head_topic"})
    
    # 최종 수집 항목을 list에 저장
    # link에 기본 url 추가
    base_url = 'https://news.naver.com'
    # 전체 토픽 title과 link 출력
    for topic in topic_list :
        try : 
            a_t = topic.find('a')
            topic_title.append(a_t.text.replace('\n',''))
            topic_link.append(base_url + a_t['href'])
        except :
            print('에러발생')
    return({"topic":topic_title, "url":topic_link,"section":section})



In [39]:
# 모든 섹션의 topic을 하나의 dataframe에 저장하기 위해 빈 dataframe을 생성
dict_sub={}
dict_sub['topic']=[]
dict_sub['url']=[]
dict_sub['section']=[]

topic_df = pd.DataFrame(dict_sub)
topic_df


,topic,url,section


In [40]:
#섹션별 헤드라인 추출
for i in range(1,7) : 
    temp = pd.DataFrame(get_topic(df_menu['link'][i],df_menu['section'][i]))
    topic_df = pd.concat([topic_df, temp], axis =0, ignore_index=True)
    
    
    

에러발생
에러발생
에러발생
에러발생
에러발생
에러발생
에러발생


In [45]:
topic_df.head(10)
# topic_df.tail(10)

,topic,url,section
0,20개의 관련뉴스 더보기,https://news.naver.com/main/clusterArticles.na...,정치
1,15개의 관련뉴스 더보기,https://news.naver.com/main/clusterArticles.na...,정치
2,36개의 관련뉴스 더보기,https://news.naver.com/main/clusterArticles.na...,정치
3,19개의 관련뉴스 더보기,https://news.naver.com/main/clusterArticles.na...,정치
4,52개의 관련뉴스 더보기,https://news.naver.com/main/clusterArticles.na...,정치
5,41개의 관련뉴스 더보기,https://news.naver.com/main/clusterArticles.na...,정치
6,53개의 관련뉴스 더보기,https://news.naver.com/main/clusterArticles.na...,정치
7,28개의 관련뉴스 더보기,https://news.naver.com/main/clusterArticles.na...,정치
8,38개의 관련뉴스 더보기,https://news.naver.com/main/clusterArticles.na...,정치
9,15개의 관련뉴스 더보기,https://news.naver.com/main/clusterArticles.na...,정치


In [42]:
#파일로 저장
topic_df.to_csv('./crawl_data/naver_news_topic.csv')

In [ ]:
d